In [110]:
import sys
sys.path.append('/Users/isabelcorpus/.pyenv/versions/3.9.0/lib/python3.9/site-packages')
import pandas as pd
import numpy as np 
import matplotlib as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from analysis_helper import *
import json
import seaborn as sns
from sentence_transformers import SentenceTransformer

In [112]:
# Prepare data: 
# load files, normalize embeddings, preprocess for analysis and modeling 
news = pd.read_csv("data/MINDsmall_train/news.tsv", sep='\t', 
                   names = ["news_id", "category", "subcategory", "title", "abstract", "url", "title_entities", "abstract_entities"])
news = process_tsv(news, ['title_entities', 'abstract_entities'])

entity_id, entity_vec = load_embeddings('data/MINDsmall_train/entity_embedding.vec')
relation_id, relation_vec = load_embeddings('data/MINDsmall_train/relation_embedding.vec')

entity_vec = normalize(entity_vec)

# extract WikiData Knowledge Graph entity IDs for title and abstract
news['title_entity_ids'] = extract_entity_list(news, 'title_entities', 'WikidataId')
news['abstract_entity_ids'] = extract_entity_list(news, 'abstract_entities', 'WikidataId')

# mean pool entity embeddings to create news embeddings  
news_embeddings = mean_pooled_news_embeddings(entity_vec, entity_id, news, "title_entity_ids")

/Users/isabelcorpus/sample_project/mind/analysis_helper/preprocessing.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col_i].fillna('{}', inplace=True)


In [ ]:
# Generate sentence-bert embeddings using pretrained SBERT model 
# load pre-trained sbert 
sbert = SentenceTransformer('all-MiniLM-L6-v2')
title_embeddings = sbert.encode(np.vstack(news.title))

# join embeddings with news metadata 
title_embedding_df = pd.DataFrame(zip(news.news_id, title_embeddings), columns = ["news_id", "title_embedding"])
news_title_entity = pd.merge(title_embedding_df, news_embeddings, how = "inner", on = "news_id")